# 索引

## 概念

Index是一种数据结构，可让我们快速检索用户查询的相关上下文。

对于 LlamaIndex 而言，它是检索增强生成 (RAG) 用例的核心基础。

从高层次上讲：

- Indexes是由文档构建的
- 用于构建查询引擎和聊天引擎，从而实现对数据的问答和聊天。

在底层：

- Indexes将数据存储在Node对象（代表原始文档的块）中
- 公开支持额外配置和自动化的Retriever接口

最常见的索引是VectorStoreIndex


## 索引是怎么工作的

基础术语：

- Node：对应于 Document 中的一段文本。LlamaIndex 接收 Document 对象并在内部将其解析/分块为 Node 对象。
- 响应合成：我们的模块根据检索到的节点合成响应。您可以看到如何 [指定不同的响应模式](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes/)。

### 摘要索引 - SummaryIndex

#### 索引结构

摘要索引只是将节点存储为顺序链。

![](https://docs.llamaindex.ai/en/stable/_static/indices/list.png)

#### 查询方式

将列表中的所有节点加载到我们的响应合成模块中。

![](https://docs.llamaindex.ai/en/stable/_static/indices/list_query.png)

提供了多种查询摘要索引的方法:

- 基于嵌入的查询（将获取前 k 个邻居）
- 添加关键字过滤器

![](https://docs.llamaindex.ai/en/stable/_static/indices/list_filter_query.png)

### 向量存储索引 - VectorStoreIndex

#### 索引结构

将每个节点和相应的嵌入存储在向量存储中。

![](https://docs.llamaindex.ai/en/stable/_static/indices/vector_store.png)


#### 查询方式

查询向量存储索引涉及获取前 k 个最相似的节点，并将它们传递到我们的响应合成模块中。

![](https://docs.llamaindex.ai/en/stable/_static/indices/vector_store_query.png)

### 树索引 - TreeIndex

#### 索引结构

树索引从一组节点（成为该树中的叶节点）构建一个层次树。

我的理解：

- 叶子节点和 SummaryIndex 中的节点是一样的
- 父节点保存总结叶节点的文本
- 创建父节点时的摘要过程是在创建索引时使用LLM完成的
- TreeIndex 包含摘要处理的提示词
- 叶子节点的数量小于一定数量，则不会构建树结构
- 具有相同源文件的叶子节点不一定具有相同的父节点
- 可能有多个根节点
- 可能存在多个父子层次结构
- 摘要结果是英语的，可以通过修改提示词语言来改变
- TreeIndex，num_children可以指定 ，默认为10，上限是20

![](https://docs.llamaindex.ai/en/stable/_static/indices/tree.png)




#### 查询方式

查询树形索引涉及从根节点向下遍历到叶节点。

默认情况下，（child_branch_factor=1），查询在给定父节点的情况下选择一个子节点。

如果child_branch_factor=2，查询每级选择两个子节点。

![](https://docs.llamaindex.ai/en/stable/_static/indices/tree_query.png)

### 关键字表索引 - KeywordTableIndex

#### 索引结构

关键字表索引从每个节点提取关键字，并建立从每个关键字到该关键字对应节点的映射。

KeywordTableIndex - 索引使用 LLM 从文本中提取关键词。

![](https://docs.llamaindex.ai/en/stable/_static/indices/keyword.png)

#### 查询方式

在查询时，我们从查询中提取相关关键字，并将其与预先提取的节点关键字进行匹配以获取相应的节点。

提取的节点被传递给我们的响应合成模块。

![](https://docs.llamaindex.ai/en/stable/_static/indices/keyword_query.png)

### 属性图索引 - Property Graph Index 

工作原理是

- 首先构建一个包含标记节点和关系的知识图谱
- 图谱的构建高度可定制，范围从让 LLM 提取其想要的任何内容，到使用严格模式提取，甚至实现您自己的提取模块。

还可以跳过创建，并使用 Neo4j 等集成连接到现有知识图。

## 使用 VectorStoreIndex

向量存储是检索增强生成 (RAG) 的关键组件，因此您最终会在使用 LlamaIndex 制作的几乎每个应用程序中直接或间接地使用它们。

### 加载数据到索引

#### 基本用法

要点：

- 使用时from_documents，您的文档将被分成块并解析为Node对象
- 默认情况下，VectorStoreIndex 将所有内容存储在内存中
- 默认情况下，VectorStoreIndex将以 2048 个节点为一批生成和插入向量，远程托管可能需要调整这个数字

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents and build index
documents = SimpleDirectoryReader(
    "../../examples/data/paul_graham"
).load_data()
index = VectorStoreIndex.from_documents(documents)

#### 使用摄取管道创建节点

想要更好地控制文档的索引方式，我们建议您使用提取管道。这允许您自定义节点的

- 分块、
- 元数据和
- 嵌入。

In [ ]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        TitleExtractor(),
        OpenAIEmbedding(),
    ]
)

# run the pipeline
nodes = pipeline.run(documents=[Document.example()])

pipeline需要设置vector_store，这样可以自动存储处理的转换。

In [ ]:
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        TitleExtractor(),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)

# Ingest directly into a vector db
pipeline.run(documents=[Document.example()])

#### 直接创建和管理节点

想要完全控制索引，您可以手动创建和定义节点并将它们直接传递给索引构造函数：

In [ ]:
from llama_index.core.schema import TextNode

node1 = TextNode(text="<text_chunk>", id_="<node_id>")
node2 = TextNode(text="<text_chunk>", id_="<node_id>")
nodes = [node1, node2]
index = VectorStoreIndex(nodes)

需要处理随时间变化的数据源。Index类具有插入，删除，更新和刷新操作，您可以在下面了解有关它们的更多信息：

- 元数据提取
- 文件管理

### 存储向量索引

LlamaIndex 支持几十个向量存储。

你可以通过传入一个来指定使用哪一个StorageContext，然后在其上指定vector_store参数。

In [ ]:
import pinecone
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.pinecone import PineconeVectorStore

# init pinecone
pinecone.init(api_key="<api_key>", environment="<environment>")
pinecone.create_index(
    "quickstart", dimension=1536, metric="euclidean", pod_type="p1"
)

# construct vector store and customize storage context
storage_context = StorageContext.from_defaults(
    vector_store=PineconeVectorStore(pinecone.Index("quickstart"))
)

# Load documents and build index
documents = SimpleDirectoryReader(
    "../../examples/data/paul_graham"
).load_data()
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### 可组合索引

（VectorStoreIndex以及任何其他索引/检索器）能够检索通用对象，包括

- 节点引用 - nodes
- 查询引擎 - query engines
- 检索器 - retrievers
- 查询管道 - query pipeline

检索到这些对象，它们将使用提供的查询自动运行：

In [ ]:
from llama_index.core.schema import IndexNode

query_engine = other_index.as_query_engine
obj = IndexNode(
    text="A query engine describing X, Y, and Z.",
    obj=query_engine,
    index_id="my_query_engine",
)

index = VectorStoreIndex(nodes=nodes, objects=[obj])
retriever = index.as_retreiver(verbose=True)

## 使用属性图索引 - PropertyGraphIndex

属性图是带有属性（即元数据）的标记节点（即实体类别、文本标签等）的知识集合。

通过关系链接在一起形成结构化路径。

在 LlamaIndex 中，PropertyGraphIndex提供了围绕

- 构建一个图，constructing a graph
- 查询一个图，querying a graph

### 使用

#### 基本使用

In [ ]:
from llama_index.core import PropertyGraphIndex

# create
index = PropertyGraphIndex.from_documents(
    documents,
)

# use
retriever = index.as_retriever(
    include_text=True,  # include source chunk with matching paths
    similarity_top_k=2,  # top k for vector kg node retrieval
)
nodes = retriever.retrieve("Test")

query_engine = index.as_query_engine(
    include_text=True,  # include source chunk with matching paths
    similarity_top_k=2,  # top k for vector kg node retrieval
)
response = query_engine.query("Test")

# save and load
index.storage_context.persist(persist_dir="./storage")

from llama_index.core import StorageContext, load_index_from_storage

index = load_index_from_storage(
    StorageContext.from_defaults(persist_dir="./storage")
)

# loading from existing graph store (and optional vector store)
# load from existing graph/vector store
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store, vector_store=vector_store, ...
)

#### 构建

LlamaIndex 中的属性图构建通过对每个块执行一系列操作kg_extractors，并将实体和关系作为元数据附加到每个 llama-index 节点来实现。

设置提取器（默认值为SimpleLLMPathExtractor和ImplicitPathExtractor）：

In [ ]:
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[extractor1, extractor2, ...],
)

# insert additional documents / nodes
index.insert(document)
index.insert_nodes(nodes)

##### SimpleLLMPathExtractor

使用 LLM 提取简短语句来提示和解析格式为 ( entity1, relation, entity2)的单跳路径

In [ ]:
from llama_index.core.indices.property_graph import SimpleLLMPathExtractor

kg_extractor = SimpleLLMPathExtractor(
    llm=llm,
    max_paths_per_chunk=10,
    num_workers=4,
    show_progress=False,
)

##### ImplicitPathExtractor

使用每个 llama-index 节点对象上的属性提取路径。

不需要 LLM 或嵌入模型来运行，因为它仅仅解析 llama-index 节点对象上已经存在的属性。

In [ ]:
from llama_index.core.indices.property_graph import ImplicitPathExtractor

kg_extractor = ImplicitPathExtractor()

##### SchemaLLMPathExtractor

按照允许的实体、关系以及哪些实体可以连接到哪些关系的严格模式提取路径。

In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

# recommended uppercase, underscore separated
entities = Literal["PERSON", "PLACE", "THING"]
relations = Literal["PART_OF", "HAS", "IS_A"]
schema = {
    "PERSON": ["PART_OF", "HAS", "IS_A"],
    "PLACE": ["PART_OF", "HAS"],
    "THING": ["IS_A"],
}

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=schema,
    strict=True,  # if false, will allow triples outside of the schema
    num_workers=4,
    max_paths_per_chunk=10,
    show_progres=False,
)

#### 检索和查询

标记属性图可以通过多种方式查询以检索节点和路径。

在 LlamaIndex 中，我们可以同时组合多种节点检索方法！

In [ ]:
# create a retriever
retriever = index.as_retriever(sub_retrievers=[retriever1, retriever2, ...])

# create a query engine
query_engine = index.as_query_engine(
    sub_retrievers=[retriever1, retriever2, ...]
)

#### 存储

支持的图存储：

- SimplePropertyGraphStore
- Neo4jPropertyGraphStore
- NebulaPropertyGraphStore
- TiDBPropertyGraphStore

##### 保存到磁盘/从磁盘加载

默认属性图存储SimplePropertyGraphStore将所有内容存储在内存中并持久保存并从磁盘加载。

In [ ]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.indices import PropertyGraphIndex

# create
index = PropertyGraphIndex.from_documents(documents)

# save
index.storage_context.persist("./storage")

# load
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)

##### 使用集成保存和加载

##### 直接使用 Property Graph Store

#### 高级定制

## 文件管理

大多数 LlamaIndex 索引结构允许插入、删除、更新和刷新操作。

### 插入

可以在初始构建索引后将新文档“插入”到任何索引数据结构中。

文档将被分解为节点并被纳入索引。

背后的底层机制取决于索引结构。例如，

- 对于摘要索引，会将新文档作为列表中的附加节点插入
- 对于向量存储索引，会将新文档（和嵌入）插入到底层文档/嵌入存储中

In [ ]:
from llama_index.core import SummaryIndex, Document

index = SummaryIndex([])
text_chunks = ["text_chunk_1", "text_chunk_2", "text_chunk_3"]

doc_chunks = []
for i, text in enumerate(text_chunks):
    doc = Document(text=text, id_=f"doc_id_{i}")
    doc_chunks.append(doc)

# insert
for doc_chunk in doc_chunks:
    index.insert(doc_chunk)

### 删除

通过指定 document_id 从大多数索引数据结构中“删除”文档。

与文档对应的所有节点都将被删除。

树索引目前不支持删除。

In [ ]:
index.delete_ref_doc("doc_id_0", delete_from_docstore=True)

`delete_from_docstore` 如果您在使用相同文档库的索引之间共享节点，则默认为False。

### 更新

如果文档已存在于索引中，则可以使用相同的文档“更新”。

传递了一些额外的 kwargs 来确保文档从 docstore 中删除。

In [ ]:
# NOTE: the document has a `doc_id` specified
doc_chunks[0].text = "Brand new document text"
index.update_ref_doc(
    doc_chunks[0],
    update_kwargs={"delete_kwargs": {"delete_from_docstore": True}},
)

### 刷新

加载数据时，如果设置每个文档的文档id_，可以自动刷新索引。

refresh()函数只会更新具有相同 doc id_，但不同文本内容的文档。索引中不存在的任何文档也将被插入。

refresh()还返回一个布尔列表，指示输入中的哪些文档已在索引中刷新。

当从不断更新新信息的目录中读取数据时，这非常有用。

id_若要在使用时自动设置文档SimpleDirectoryReader，您可以设置filename_as_id标志。

In [ ]:
# modify first document, with the same doc_id
doc_chunks[0] = Document(text="Super new document text", id_="doc_id_0")

# add a new document
doc_chunks.append(
    Document(
        text="This isn't in the index yet, but it will be soon!",
        id_="doc_id_3",
    )
)

# refresh the index
refreshed_docs = index.refresh_ref_docs(
    doc_chunks, update_kwargs={"delete_kwargs": {"delete_from_docstore": True}}
)

# refreshed_docs[0] and refreshed_docs[-1] should be true

### 文档追踪

任何使用文档存储的索引（即除了大多数向量存储集成之外的所有索引），您还可以看到已插入文档存储的文档。

输出中的每个条目都将摄取的文档显示id_为键，并将其node_ids与分割成的节点关联。

最后，metadata还会跟踪每个输入文档的原始词典。您可以在自定义文档metadata中阅读有关该属性的更多信息。

In [ ]:
print(index.ref_doc_info)
"""
> {'doc_id_1': RefDocInfo(node_ids=['071a66a8-3c47-49ad-84fa-7010c6277479'], metadata={}),
   'doc_id_2': RefDocInfo(node_ids=['9563e84b-f934-41c3-acfd-22e88492c869'], metadata={}),
   'doc_id_0': RefDocInfo(node_ids=['b53e6c2f-16f7-4024-af4c-42890e945f36'], metadata={}),
   'doc_id_3': RefDocInfo(node_ids=['6bedb29f-15db-4c7c-9885-7490e10aa33f'], metadata={})}
"""

## LlamaCloudIndex

LlamaCloud 是新一代托管解析、提取和检索服务，旨在为您的 LLM 和 RAG 应用程序带来生产级上下文增强。

目前，LlamaCloud 支持

- 托管摄取 API，处理解析和文档管理
- 托管检索 API，为您的 RAG 系统配置最佳检索

## 元数据提取

### 介绍

在许多情况下，尤其是长文档中，一段文本可能缺少必要的上下文来将该段文本与其他类似的文本段区分开来。

为了解决这个问题，我们使用 LLM 来提取与文档相关的某些上下文信息，以便更好地帮助检索和语言模型消除看起来相似的段落的歧义。

### 用法

首先，我们定义一个元数据提取器，它接收将按顺序处理的特征提取器列表。

然后我们将其提供给节点解析器，它将向每个节点添加额外的元数据。

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
)
from llama_index.extractors.entity import EntityExtractor

transformations = [
    SentenceSplitter(),
    TitleExtractor(nodes=5),
    QuestionsAnsweredExtractor(questions=3),
    SummaryExtractor(summaries=["prev", "self"]),
    KeywordExtractor(keywords=10),
    EntityExtractor(prediction_threshold=0.5),
]

然后，我们可以在输入文档或节点上运行转换：

In [ ]:
from llama_index.core.ingestion import IngestionPipeline

pipeline = IngestionPipeline(transformations=transformations)

nodes = pipeline.run(documents=documents)

以下是提取的元数据的示例：

In [ ]:
{'page_label': '2',
 'file_name': '10k-132.pdf',
 'document_title': 'Uber Technologies, Inc. 2019 Annual Report: Revolutionizing Mobility and Logistics Across 69 Countries and 111 Million MAPCs with $65 Billion in Gross Bookings',
 'questions_this_excerpt_can_answer': '\n\n1. How many countries does Uber Technologies, Inc. operate in?\n2. What is the total number of MAPCs served by Uber Technologies, Inc.?\n3. How much gross bookings did Uber Technologies, Inc. generate in 2019?',
 'prev_section_summary': "\n\nThe 2019 Annual Report provides an overview of the key topics and entities that have been important to the organization over the past year. These include financial performance, operational highlights, customer satisfaction, employee engagement, and sustainability initiatives. It also provides an overview of the organization's strategic objectives and goals for the upcoming year.",
 'section_summary': '\nThis section discusses a global tech platform that serves multiple multi-trillion dollar markets with products leveraging core technology and infrastructure. It enables consumers and drivers to tap a button and get a ride or work. The platform has revolutionized personal mobility with ridesharing and is now leveraging its platform to redefine the massive meal delivery and logistics industries. The foundation of the platform is its massive network, leading technology, operational excellence, and product expertise.',
 'excerpt_keywords': '\nRidesharing, Mobility, Meal Delivery, Logistics, Network, Technology, Operational Excellence, Product Expertise, Point A, Point B'}

### 自定义提取器

如果提供的提取器不能满足您的需求，您还可以定义自定义提取器，如下所示：

In [ ]:
from llama_index.core.extractors import BaseExtractor


class CustomExtractor(BaseExtractor):
    async def aextract(self, nodes) -> List[Dict]:
        metadata_list = [
            {
                "custom": node.metadata["document_title"]
                + "\n"
                + node.metadata["excerpt_keywords"]
            }
            for node in nodes
        ]
        return metadata_list

extractor.extract()将自动aextract()在后台调用，以提供同步和异步入口点。

在更高级的示例中，它还可以利用llm从节点内容和现有元数据中提取特征。